In [88]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import GridSearchCV

df=pd.read_csv(r"C:\nydsa bootcamp slides\Projects\4\diabetic_data.csv")
origin_df=df.copy()

In [89]:
#Check all column names
origin_df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

### 1 . Remove weight column

In [90]:
origin_df.drop('weight',axis=1, inplace=True)

In [91]:
origin_df.shape

(101766, 49)

### 2. Remove 3 rows that "gender"="Unknow/Invalid value"

In [92]:
origin_df.drop(origin_df[origin_df['gender']=="Unknown/Invalid"].index,axis=0,inplace=True)

In [93]:
origin_df.shape

(101763, 49)

### 3. Drop 2273 rows that "race"="?"

In [94]:
origin_df.drop(origin_df[origin_df['race']=="?"].index,axis=0,inplace=True)

In [95]:
origin_df.shape

(99492, 49)

### 4. Drop column "payer_code"

In [96]:
origin_df.drop('payer_code',axis=1, inplace=True)

In [97]:
origin_df.shape

(99492, 48)

### 5. Replace "?" value with "unknown" in column "medical_specialty", 48766 rows

In [98]:
origin_df['medical_specialty']=origin_df['medical_specialty'].replace("?","unknown")

In [99]:
origin_df[origin_df['medical_specialty']=="?"]

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted


### 6. remove rows that are ? in diag_1, diag_2 and diag_3
* originally diag_1 has 21 ? 
* diag_2 has 358 ?
* diag_3 has 1423 ?

Some rows may have both diag_1 and diag_2 as ? , also some rows with ? values got removed in the first 5 steps. 
Therefore, in total in this step we have 1440 rows removed

In [100]:
origin_df.drop(origin_df[origin_df['diag_1']=="?"].index,axis=0,inplace=True)

In [101]:
origin_df.drop(origin_df[origin_df['diag_2']=="?"].index,axis=0,inplace=True)

In [102]:
origin_df.drop(origin_df[origin_df['diag_3']=="?"].index,axis=0,inplace=True)

In [103]:
origin_df.shape

(98052, 48)

### 7. remove the following columns because of too few real data
* 'tolbutamide', 23 records
* 'glipizide-metformin', 13 records
* 'acetohexamide', 1 record
* 'troglitazone', 3 record
* 'examide', 0 record
* 'citoglipton', 0 record
* 'glimepiride-pioglitazone', 1 record
* 'metformin-rosiglitazone', 2 records
* 'metformin-pioglitazone', 1 record

In [104]:
origin_df.drop(['tolbutamide','glipizide-metformin','acetohexamide','troglitazone','examide',
                'citoglipton','glimepiride-pioglitazone','metformin-rosiglitazone','metformin-pioglitazone'],axis=1, inplace=True)

In [105]:
origin_df.shape

(98052, 39)

### 8. Based on our discussion , remove "discharge_disposition_id"= "expired" rows (id = 11,19,20,21)
* 11 - 1606 rows
* 19 - 8 rows
* 20 - 2 rows
* 21 - 0 rows

In [106]:
origin_df.drop(origin_df[origin_df['discharge_disposition_id']==11].index,axis=0,inplace=True)

In [107]:
origin_df.drop(origin_df[origin_df['discharge_disposition_id']==19].index,axis=0,inplace=True)

In [108]:
origin_df.drop(origin_df[origin_df['discharge_disposition_id']==20].index,axis=0,inplace=True)

In [109]:
origin_df.shape

(96436, 39)

### 9. Based on the paper, remove " discharge_disposition_id"="discharge to hospice" (id=13,14)
* 13-396 rows
* 14-368 rows

In [110]:
origin_df.drop(origin_df[origin_df['discharge_disposition_id']==13].index,axis=0,inplace=True)
origin_df.drop(origin_df[origin_df['discharge_disposition_id']==14].index,axis=0,inplace=True)

In [111]:
origin_df.shape

(95672, 39)

### 10. drop "encounter_id" and "patient_nbr" column
* in my understanding these are just  unique IDs for the patients

In [112]:
origin_df.drop(['encounter_id','patient_nbr'],axis=1, inplace=True)

In [113]:
origin_df.shape

(95672, 37)

### 11. update "readmitted" column to "yes" or "no"
* "<30" replace with "yes"
* all others replace with "no"

In [114]:
origin_df['readmitted']=origin_df['readmitted'].replace(["<30",">30"],["Yes","NO"])

In [115]:
set(origin_df['readmitted'])

{'NO', 'Yes'}

### 12. We have around 800 different categories in each diag_ column, combine them to the 9 main categories based on the paper

In [116]:
len(set(origin_df['diag_1']))

711

In [118]:
# The replace_diag function can be used to replace the ids in the diag_1,diag_2 and diag_3 to the 9 main categories
def replace_diag(df):
    Circulatory_list = [str(i) for i in range(350,460)]
    Respiratory_list = [str(i) for i in range(460,520)]
    Digestive_list = [str(i) for i in range(520,589)]    
    Injury_list = [str(i) for i in range(800,1000)]
    Musculoskeletal_list=[str(i) for i in range(710,740)]
    Genitourinary_list=[str(i) for i in range(589,630)]
    Neoplasms_list=[str(i) for i in range(140,240)]
    
    for i in df:
        if  (i in Circulatory_list) or i=='785':
            df.replace(i,'Circulatory',inplace=True)
        elif (i in Respiratory_list) or i =='786':
            df.replace(i,'Respiratory', inplace=True)
        elif (i in Digestive_list) or i =='787':
            df.replace(i,'Digestive', inplace=True)
        elif (i in Injury_list):
            df.replace(i,'Injury', inplace=True)
        elif (i in Musculoskeletal_list):
            df.replace(i,'Musculoskeletal', inplace=True)
        elif (i in Genitourinary_list) or i =='788':
            df.replace(i,'Genitourinary', inplace=True)
        elif (i in Neoplasms_list):
            df.replace(i,'Neoplasms', inplace=True)
        elif len(i)>3:
            df.replace(i,'Diabetes', inplace=True)
        else:
            df.replace(i,'Other',inplace=True)
            

In [120]:
replace_diag(origin_df['diag_1'])
len(set(origin_df['diag_1']))

9

In [121]:
replace_diag(origin_df['diag_2'])
len(set(origin_df['diag_2']))

9

In [122]:
replace_diag(origin_df['diag_3'])
len(set(origin_df['diag_3']))

9

### Okay, we get our origin dataset, save it as "origin.csv" first. Then we move to encode the nominal columns 

In [158]:
origin_df.shape

(95672, 37)

In [125]:
origin_df.to_csv("origin.csv")

* **


### 12. Encode the following columns for xgboost 
race,gender,age,medical_specialty,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,
chlorpropamide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,acarbose,miglitol,tolazamide,insulin,
glyburide-metformin,change,diabetesMed

In [187]:
#copy origin_df to encoder_df, so we don't make changes on origin_df
encoder_df=pd.read_csv('origin.csv',index_col=0)
lencoder=LabelEncoder()
ind="_labels"
encoder_list=['race','gender','age','medical_specialty','max_glu_serum','A1Cresult','metformin','repaglinide','nateglinide',
        'chlorpropamide','glimepiride','glipizide','glyburide','pioglitazone','rosiglitazone','acarbose','miglitol',
        'tolazamide','insulin','glyburide-metformin','change','diabetesMed','diag_1', 'diag_2', 'diag_3']

numeric_col=['admission_type_id','discharge_disposition_id','admission_source_id','time_in_hospital','num_lab_procedures',
            'num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses',
            'readmitted']

In [188]:
origin_df.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'medical_specialty', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'tolazamide',
       'insulin', 'glyburide-metformin', 'change', 'diabetesMed',
       'readmitted'],
      dtype='object')

In [190]:
features=lencoder.fit_transform(encoder_df['race']).reshape((-1,1))
for i in range(1,len(encoder_list)):
    x=lencoder.fit_transform(encoder_df[encoder_list[i]]).reshape((-1,1))
    features=np.concatenate((features,x),axis=1)

In [191]:
temp_df=pd.DataFrame(features,columns=encoder_list)
temp_df.index=encoder_df.index

In [195]:
combined_df=pd.concat([temp_df,encoder_df[numeric_col]],axis=1)
combined_df


,race,gender,age,medical_specialty,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,...,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
1,2,0,1,72,2,2,1,1,1,1,...,7,3,59,0,18,0,0,0,9,NO
2,0,0,2,72,2,2,1,1,1,1,...,7,2,11,5,13,2,0,1,6,NO
3,2,1,3,72,2,2,1,1,1,1,...,7,2,44,1,16,0,0,0,7,NO
4,2,1,4,72,2,2,1,1,1,1,...,7,1,51,0,8,0,0,0,5,NO
5,2,1,5,72,2,2,1,1,1,1,...,2,3,31,6,16,0,0,0,9,NO
6,2,1,6,72,2,2,2,1,1,1,...,2,4,70,1,21,0,0,0,7,NO
7,2,1,7,72,2,2,1,1,1,1,...,7,5,73,0,12,0,0,0,8,NO
8,2,0,8,72,2,2,1,1,1,1,...,4,13,68,2,28,0,0,0,8,NO
9,2,0,9,18,2,2,1,1,1,1,...,4,12,33,3,18,0,0,0,8,NO
10,0,0,4,72,2,2,1,1,1,1,...,7,9,47,2,17,0,0,0,9,NO


In [211]:
combined_df.to_csv('encoded.csv')

### Split the final encoded dataset into 80%-20% train-test datasets with seed (1)

In [208]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(combined_df.iloc[:,:36], combined_df['readmitted'], test_size=0.2,random_state=1)